# 1. Pre-reqs
We may need to install the rrscale and memnorm packages

In [3]:
#install.packages('../r_packages/rrscale_rpkg/rrscale_1.0.3.tar.gz', repos=NULL,type="source")
#install.packages('../r_packages/mema_norm_rpkg/memanorm_0.0.0.9004.tar.gz', repos=NULL,type="source")

Installing package into ‘/home/greg/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)

Installing package into ‘/home/greg/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [2]:
library('rrscale')
library('memanorm')
source('processing_scripts/util.R')


Attaching package: ‘memanorm’


The following object is masked from ‘package:rrscale’:

    svdc




and then we'll set up a directory for the processed data

In [3]:
# identifying tag for the data
tag = "MCF10A"

# directory to put processed data
ddir ='processed_data/'%+%tag%+%"/"
dir.create(ddir,showWarning=FALSE,recursive=TRUE)

Now let's make a list containing the feature matrices 

In [4]:
source('processing_scripts/make_data_matrices.R') # helper script
tsv_dir = 'raw_data/'%+%tag%+%'/'
data = make_data_matrices(tsv_dir)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following object is masked _by_ ‘.GlobalEnv’:

    .N


The following objects are masked from ‘package:dplyr’:

    between, first, last




here each feature matrix is has a row for each well and a column for each spot. We can look at the dimension of the first few feature matrices

In [5]:
head(lapply(data,dim))

$WellIndex
[1] 192 667

$ArrayRow
[1] 192 667

$ArrayColumn
[1] 192 667

$Barcode
[1] 192 667

$Well
[1] 192 667

$Spot
[1] 192 667

we'll make the list of features we want to RR transform

In [6]:
to_rr = grep("_PA_|_CP_",names(data),value=TRUE)
to_rr = to_rr[!grepl("_SE",to_rr)]
unum = sapply(to_rr,function(x)length(unique(unlist(data[[x]]))))
to_rr = names(unum[unum>20])
sample(to_rr,5)

[1] "Nuclei_CP_AreaShape_Extent"                     
[2] "Cytoplasm_CP_AreaShape_Area"                    
[3] "Nuclei_CP_Texture_Contrast_Fibrillarin_3_0"     
[4] "Cytoplasm_CP_Intensity_IntegratedIntensity_Dapi"
[5] "Nuclei_CP_AreaShape_Orientation"

and then save the feature matrices and list of features

In [7]:
dl = list(data=data,rr_feats = to_rr)
saveRDS(dl,ddir%+%tag%+%'.rds')

now we can apply the RR transformation to the matrices (this can take a while)

# 2. RRScale

In [8]:
source('processing_scripts/estimate.R',chdir=TRUE)
scaledY_file = ddir%+%"scaledY_"%+%tag%+%".rds"
if(file.exists(scaledY_file)){
    cat("Reading from cache.\n")
    scaledY = readRDS(scaledY_file)
} else {
    cat("No cache, re-running.\n")
    scaledY = scale_data(dl)
    saveRDS(scaledY,file=scaledY_file)
}

Reading from cache.


Finally, we'll print the session info for reproducibility

# 3. Clean up spatial effects

In [9]:
base_fmats = dl$data
rr_fmats = lapply(scaledY,"[[","RR")

ecmps = factor(unlist(base_fmats$ECMp[1,]))
ligands = factor(unlist(base_fmats$Ligand[,1]))

find which features are measured in all batches for normalization (the "dapi" features)

In [10]:
all_missing = sapply(rr_fmats,function(y)apply(y,1,function(x)all(!is.finite(x))))
all_batches = apply(!all_missing,2,all)
all_batches = names(all_batches[all_batches])
all_batches = all_batches[!grepl("Loess",all_batches)]
all_batches

[1] "Nuclei_CP_AreaShape_Area"                    
 [2] "Nuclei_CP_AreaShape_Compactness"             
 [3] "Nuclei_CP_AreaShape_Eccentricity"            
 [4] "Nuclei_CP_AreaShape_Extent"                  
 [5] "Nuclei_CP_AreaShape_FormFactor"              
 [6] "Nuclei_CP_AreaShape_MajorAxisLength"         
 [7] "Nuclei_CP_AreaShape_MaxFeretDiameter"        
 [8] "Nuclei_CP_AreaShape_MaximumRadius"           
 [9] "Nuclei_CP_AreaShape_MeanRadius"              
[10] "Nuclei_CP_AreaShape_MedianRadius"            
[11] "Nuclei_CP_AreaShape_MinFeretDiameter"        
[12] "Nuclei_CP_AreaShape_MinorAxisLength"         
[13] "Nuclei_CP_AreaShape_Orientation"             
[14] "Nuclei_CP_AreaShape_Perimeter"               
[15] "Nuclei_CP_AreaShape_Solidity"                
[16] "Nuclei_CP_Intensity_IntegratedIntensity_Dapi"
[17] "Nuclei_CP_Intensity_MedianIntensity_Dapi"    
[18] "Nuclei_PA_AreaShape_Neighbors"               
[19] "Spot_PA_SpotCellCount"                       
[20] "Nuclei_PA_Cycle_DNA2NProportion"             
[21] "Nuclei_PA_Cycle_DNA4NProportion"

In [53]:
#devtools::load_all('../r_packages/mema_norm_rpkg/memanorm')
d_none = 0
unadj_norm_out = remove_spatial(f_mats = rr_fmats,
                  adj_idx = all_batches,
                  row_rep = ligands,
                  col_rep = ecmps,
                  d=d_none,
                  verbose=FALSE)
unadj_fmats=unadj_norm_out$f_adj
names(unadj_fmats) <- names(unadj_fmats)%+%"_RR"
saveRDS(unadj_fmats,ddir%+%"unadjusted_rr_"%+%d_none%+%"_"%+%tag%+%".rds")

In [48]:
library('memanorm')
d_adj=15
norm_out = remove_spatial(f_mats = rr_fmats,
                  adj_idx = all_batches,
                  row_rep = ligands,
                  col_rep = ecmps,
                  d=d_adj,
                  verbose=FALSE)
adj_fmats=norm_out$f_adj
names(adj_fmats) <- names(adj_fmats)%+%"_RR_ADJ"
saveRDS(adj_fmats,ddir%+%"adjusted_rr_"%+%d_adj%+%"_"%+%tag%+%".rds")


Imputing [>-------------------------------------------------------------------------] 2/116 (  2%) elapsed:  3s eta:  3m

Imputing [=>------------------------------------------------------------------------] 3/116 (  3%) elapsed:  4s eta:  3m

Imputing [==>-----------------------------------------------------------------------] 4/116 (  3%) elapsed:  5s eta:  3m

Imputing [==>-----------------------------------------------------------------------] 5/116 (  4%) elapsed:  6s eta:  2m

Imputing [===>----------------------------------------------------------------------] 6/116 (  5%) elapsed:  8s eta:  2m

Imputing [===>----------------------------------------------------------------------] 7/116 (  6%) elapsed:  9s eta:  2m

Imputing [====>---------------------------------------------------------------------] 8/116 (  7%) elapsed: 10s eta:  2m

Imputing [=====>--------------------------------------------------------------------] 9/116 (  8%) elapsed: 11s eta:  2m

Imputing [=====>-------

unadjusted features (same exact procedure but not removing spatial)

# 4. make .csv data matrices

In [49]:
dmsn = dim(base_fmats[[1]])
fmat_col = array(rep(1:dmsn[2],each=dmsn[1]),dmsn)
fmat_row = t(array(rep(1:dmsn[1],each=dmsn[2]),rev(dmsn)))
fmat_dims = list("fcol"=fmat_col,"frow"=fmat_row)

In [50]:
library('data.table')
all_fmats = c(base_fmats,unadj_fmats,adj_fmats,fmat_dims)
all_fmats = lapply(all_fmats,data.table)

In [51]:
ul_all_fmats = lapply(all_fmats,unlist)
d = data.frame(ul_all_fmats)

In [52]:
dir.create(ddir%+%"csv/",showWarnings=FALSE,recursive=TRUE)
write.csv(x=d,file=ddir%+%"csv/"%+%tag%+%"_"%+%d_adj%+%"_df.csv")
write.csv(names(adj_fmats),file=ddir%+%"csv/"%+%tag%+%"_"%+%d_adj%+%"_features.csv")